<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Data-Visualization" data-toc-modified-id="Data-Visualization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Visualization</a></span></li><li><span><a href="#Neural-Network-models" data-toc-modified-id="Neural-Network-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Neural Network models</a></span><ul class="toc-item"><li><span><a href="#Model---1" data-toc-modified-id="Model---1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Model - 1</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import pydicom
from sklearn.metrics import roc_auc_score
from copy import deepcopy
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Data Preprocessing

In [2]:
ts = pd.read_csv('/Users/rt/Desktop/College/Cancer Project/datasets/TrueSet.csv')
#display(ts)

In [3]:
#True Set

ts.drop(columns = ['Patient_id', 'Slice_id', 'Angles_Distance', 'ASM'], inplace = True)
#display(ts)

,Contrast,Dissimilarities,Homogeneity,Energy,Correlation,status,Entropy
0,33.624173,0.254810,0.991273,0.988252,0.971235,1.0,0.133671
1,52.397355,0.335625,0.991312,0.988021,0.955348,1.0,0.133671
2,32.395772,0.246661,0.991358,0.988267,0.972286,1.0,0.133671
3,57.237616,0.354848,0.991172,0.987990,0.951223,1.0,0.133671
4,84.013533,0.476040,0.990650,0.987728,0.928407,1.0,0.133671
...,...,...,...,...,...,...,...
1315,74.172211,0.996355,0.979663,0.979285,0.663702,1.0,0.279801
1316,82.000896,1.083436,0.978642,0.978261,0.629656,1.0,0.279801
1317,78.950834,1.052204,0.978875,0.978393,0.644799,1.0,0.279801
1318,83.111228,1.067657,0.978810,0.978369,0.624641,1.0,0.279801


In [4]:
#True True Set

tts = pd.read_csv('/Users/rt/Desktop/College/Cancer Project/datasets/TrueTrueSet.csv')
#display(tts)

In [5]:
tts.drop(columns = ['Patient_id', 'Slice_id', 'Angles', 'Distance', 'ASM'], inplace = True)
#display(tts)

In [6]:
#False Set

fs = pd.read_csv('/Users/rt/Desktop/College/Cancer Project/datasets/FalseSet.csv')
#display(fs)

In [7]:
fs.drop(columns = ['Patient_id', 'Slice_id', 'Angles', 'Distance', 'ASM'], inplace = True)
#display(fs)

In [8]:
#False false Set

ffs = pd.read_csv('/Users/rt/Desktop/College/Cancer Project/datasets/FalseFalseSet.csv')
#display(ffs)

In [9]:
ffs.drop(columns = ['Patient_id', 'Slice_id', 'Angles', 'Distance', 'ASM'], inplace = True)
#display(ffs)

In [10]:
#False test

ft = pd.read_csv('/Users/rt/Desktop/College/Cancer Project/datasets/FalseTest.csv')
#display(ft)

In [11]:
ft['status'] = 0
temp = ft['Entropy'].values.tolist()
ft.drop(columns = ['Patient_id', 'Slice_id', 'Angles', 'Distance', 'ASM', 'Entropy'], inplace = True)
ft['status'] = 1
ft['Entropy'] = temp
#display(ft)

In [12]:
df = pd.concat([ts, tts, fs, ffs, ft])
df = shuffle(df)
y = df['status']
df.reset_index(inplace = True, drop = True)
df.drop(columns = ['status'], inplace = True)

In [13]:
#display(df)

In [14]:
cols = df.columns
scalar = StandardScaler()
temp = scalar.fit_transform(df)
df = pd.DataFrame(temp, columns = cols)
#display(df)

# Data Visualization

In [15]:
temp = df.columns.values.tolist()
'''for i in temp:
    x = df[i]
    plt.scatter(x, y)
    plt.xlabel(i)
    plt.ylabel('Cancer presence')
    plt.title('%s vs Cancer presence'%i)
    plt.show()'''

"for i in temp:\n    x = df[i]\n    plt.scatter(x, y)\n    plt.xlabel(i)\n    plt.ylabel('Cancer presence')\n    plt.title('%s vs Cancer presence'%i)\n    plt.show()"

In [16]:
#sns.pairplot(df)

In [17]:
#plt.figure(figsize = (15, 7))
#sns.heatmap(df.corr(), annot = True)

In [18]:
def performance_metrics(y_test, y_pred):
    print('Performance metrics:')
    print(classification_report(y_test, y_pred), '\n')
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred),'\n')
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print('True positive: ', tp)
    print('True negavtive: ', tn)
    print('False positive: ', fp)
    print('False negative: ', fn)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

# Neural Network models

## Model - 1

In [20]:
classifier = Sequential()

In [21]:
classifier.add(Dense(6, activation = 'relu', 
                     kernel_initializer = 'random_normal', 
                     input_dim = 6))

In [22]:
classifier.add(Dense(6, activation = 'relu',
                    kernel_initializer = 'random_normal'))

In [23]:
classifier.add(Dense(1, activation = 'sigmoid',
                    kernel_initializer = 'random_normal'))

In [24]:
classifier.compile(optimizer = 'adam',
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

In [25]:
classifier.fit(x_train, y_train, batch_size = 5, epochs = 100)

Train on 3532 samples
Epoch 1/100
3532/3532 [==============================] - 1s 351us/sample - loss: 0.5000 - accuracy: 0.7486
Epoch 2/100
3532/3532 [==============================] - 1s 249us/sample - loss: 0.3455 - accuracy: 0.8502
Epoch 3/100
3532/3532 [==============================] - 1s 256us/sample - loss: 0.3370 - accuracy: 0.8497
Epoch 4/100
3532/3532 [==============================] - 1s 254us/sample - loss: 0.3311 - accuracy: 0.8485
Epoch 5/100
3532/3532 [==============================] - 1s 250us/sample - loss: 0.3244 - accuracy: 0.8494
Epoch 6/100
3532/3532 [==============================] - 1s 249us/sample - loss: 0.3172 - accuracy: 0.8494
Epoch 7/100
3532/3532 [==============================] - 1s 258us/sample - loss: 0.3083 - accuracy: 0.8488
Epoch 8/100
3532/3532 [==============================] - 1s 255us/sample - loss: 0.2975 - accuracy: 0.8485
Epoch 9/100
3532/3532 [==============================] - 1s 264us/sample - loss: 0.2853 - accuracy: 0.8491
Epoch 10/100
35

In [26]:
print('Accuracy on training set : ', 
      classifier.evaluate(x_train, y_train)[1])
print()
print('Loss on training set : ', 
      classifier.evaluate(x_train, y_train)[0])

3532/3532 [==============================] - 0s 48us/sample - loss: 0.1622 - accuracy: 0.9153
Accuracy on training set :  0.91534543

3532/3532 [==============================] - 0s 18us/sample - loss: 0.1622 - accuracy: 0.9153
Loss on training set :  0.16224386640070249


In [27]:
print('Accuracy on testing set : ', 
      classifier.evaluate(x_test, y_test)[1])
print()
print('Loss on testing set : ', 
      classifier.evaluate(x_test, y_test)[0])

884/884 [==============================] - 0s 20us/sample - loss: 0.1561 - accuracy: 0.9174
Accuracy on testing set :  0.9174208

884/884 [==============================] - 0s 21us/sample - loss: 0.1561 - accuracy: 0.9174
Loss on testing set :  0.15614652303279256


In [28]:
y_pred = classifier.predict_classes(x_test)
performance_metrics(y_test, y_pred)

Performance metrics:
              precision    recall  f1-score   support

         0.0       0.94      0.85      0.89       359
         1.0       0.91      0.96      0.93       525

    accuracy                           0.92       884
   macro avg       0.92      0.91      0.91       884
weighted avg       0.92      0.92      0.92       884
 

Confusion Matrix:
[[306  53]
 [ 20 505]] 

True positive:  505
True negavtive:  306
False positive:  53
False negative:  20


In [29]:
f = pd.read_csv('/Users/rt/Desktop/Test Cases March/F.csv')
ff = pd.read_csv('/Users/rt/Desktop/Test Cases March/FF.csv')
t = pd.read_csv('/Users/rt/Desktop/Test Cases March/T.csv')
tt = pd.read_csv('/Users/rt/Desktop/Test Cases March/TT.csv')

In [30]:
df1 = pd.concat([f, ff, t, tt])
df1 = shuffle(df1)
df1.drop(columns = ['Patient_id', 'Slice_id', 'Angles', 'Distance', 'ASM'], inplace = True)
y = df1['status']
df1.reset_index(inplace = True, drop = True)
df1.drop(columns = ['status'], inplace = True)

In [31]:
display(df1)

,Contrast,Dissimilarities,Homogeneity,Energy,Correlation,Entropy
0,2348.729058,23.490427,0.285105,0.202855,0.675042,6.436462
1,107.478601,0.545794,0.994833,0.994056,0.781285,0.067924
2,3078.305955,29.386245,0.260502,0.193402,0.608126,6.492836
3,67.515010,0.372724,0.995371,0.994448,0.862069,0.067924
4,2761.522072,27.796609,0.266190,0.193383,0.616940,6.455793
...,...,...,...,...,...,...
235,2309.551511,23.295056,0.281967,0.202718,0.677947,6.455793
236,2021.604887,19.974249,0.296981,0.206641,0.725121,6.476691
237,3155.580926,30.069177,0.260124,0.193241,0.593802,6.498714
238,328.435184,5.713531,0.605793,0.246213,0.943469,4.329832


In [36]:
classifier.predict(df1)

array([[0.00000000e+00],
       [8.94069672e-08],
       [0.00000000e+00],
       [4.03821468e-05],
       [0.00000000e+00],
       [7.19249249e-04],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [2.23847330e-02],
       [1.90734863e-06],
       [0.00000000e+00],
       [3.62515450e-04],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [5.69086671e-02],
       [2.17914581e-04],
       [0.00000000e+00],
       [4.13209200e-04],
       [3.61752510e-02],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [8.96477997e-02],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [2.77161598e-06],
       [0.00000000e+00],
       [0.00000000e+00],
